In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from ltn_imp.automation.knowledge_base import KnowledgeBase
from ltn_imp.parsing.parser import convert_to_ltn

## Data Prep

In [2]:
nr_samples = 100
dataset = torch.rand((nr_samples, 2))
labels_dataset = torch.sum(torch.square(dataset - torch.tensor([.5, .5])), dim=1) < .09

In [3]:
train_data = pd.read_csv("datasets/iris_training.csv")
test_data = pd.read_csv("datasets/iris_test.csv")

train_labels = train_data.pop("species")
test_labels = test_data.pop("species")

train_data = torch.tensor(train_data.to_numpy()).float()
test_data = torch.tensor(test_data.to_numpy()).float()
train_labels = torch.tensor(train_labels.to_numpy()).long()
test_labels = torch.tensor(test_labels.to_numpy()).long()

In [4]:
class BinaryDataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]
            yield data, labels
            
binary_train_loader = BinaryDataLoader(dataset[:50], labels_dataset[:50], batch_size=64, shuffle=True)
binary_test_loader = BinaryDataLoader(dataset[50:], labels_dataset[50:], batch_size=64, shuffle=False)

binary_train_loader_2 = BinaryDataLoader(train_data, train_labels, batch_size=64, shuffle=True)

In [5]:
class MultiLabelDataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.data.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            data = self.data[idxlist[start_idx:end_idx]]
            labels = self.labels[idxlist[start_idx:end_idx]]

            yield data, labels

multi_train_loader = MultiLabelDataLoader(train_data, train_labels, 64, shuffle=True)
multi_test_loader = MultiLabelDataLoader(test_data, test_labels, 64, shuffle=False)

## Classes and Methods

In [6]:
import torch

class BinaryClassifier(torch.nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.elu = torch.nn.ELU()
        self.layer1 = torch.nn.Linear(2, 16)
        self.layer2 = torch.nn.Linear(16, 16)
        self.layer3 = torch.nn.Linear(16, 2) 

    def forward(self, x):
        x = self.elu(self.layer1(x))
        x = self.elu(self.layer2(x))
        logits = self.layer3(x)
        return logits

# we define predicate P
class MultiLabelClassifier(torch.nn.Module):
    def __init__(self, layer_sizes=(4, 16, 16, 8, 3)):
        super(MultiLabelClassifier, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits

class LogitsToPredicate(torch.nn.Module):
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, y, training=True):
        logits = self.logits_model(x)
        probs = self.softmax(logits)
        # y is expected to be a one-hot encoded vector
        out = torch.sum(probs * y, dim=1, keepdim=True)
        return out

In [7]:
def compute_accuracy(loader, model):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = model.logits_model(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)
    return mean_accuracy / len(loader)

## Optimization and Evaluation

In [8]:
binary = LogitsToPredicate( BinaryClassifier() )  # type: ignore
multi = LogitsToPredicate( MultiLabelClassifier() ) # type: ignore

predicates = {"Binary": binary, "Multi": multi}
expression_1 = "( all x. (Binary(x,y)) and (all a. ( Multi(a, b))) )"
rules = [expression_1]

In [9]:
rule_to_data_loader_mapping = {expression_1: [ binary_train_loader, multi_train_loader ]}

In [10]:
loader_to_variable = {binary_train_loader: "x", multi_train_loader: "a"}

In [11]:
loader_to_target = {binary_train_loader: "y", multi_train_loader: "b"}

In [12]:
loader_classes = {binary_train_loader: 2, multi_train_loader: 3}

In [13]:
kb = KnowledgeBase(rules=rules, 
                   predicates=predicates,
                   rule_to_data_loader_mapping=rule_to_data_loader_mapping,
                   loader_to_variable=loader_to_variable,
                    loader_to_target=loader_to_target, # type: ignore
                   loader_classes=loader_classes,  # type: ignore
                   quantifier_impls={"forall" : "pmean_error"})

variables: [Variable('x'), Variable('y')]

expression: Binary(x,y)

variables: [Variable('a'), Variable('b')]

expression: Multi(a,b)

Shape: torch.Size([16, 2])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([16])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([16, 16])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([16])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([2, 16])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([2])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([16, 4])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([16])
Requires Grad: True
--------------------------------------------------
Shape: torch.Size([16, 16])
Requires Grad: True
---------------------------------------------

In [15]:
compute_accuracy(binary_train_loader, binary)

0.16

In [14]:
compute_accuracy(binary_test_loader, binary)

0.14

In [17]:
compute_accuracy(multi_train_loader, multi)

0.2979910714285714

In [16]:
compute_accuracy(multi_test_loader, multi)

0.4666666666666667

In [18]:
kb.optimize(num_epochs=1000, log_steps=100)

Epoch 1/1000, Loss: 0.6705133318901062

Epoch 101/1000, Loss: 0.4612553119659424

Epoch 201/1000, Loss: 0.4347298741340637

Epoch 301/1000, Loss: 0.4240608215332031

Epoch 401/1000, Loss: 0.4112488031387329

Epoch 501/1000, Loss: 0.3630741834640503

Epoch 601/1000, Loss: 0.2840534448623657

Epoch 701/1000, Loss: 0.2212902307510376

Epoch 801/1000, Loss: 0.18833261728286743

Epoch 901/1000, Loss: 0.16852617263793945



In [20]:
compute_accuracy(binary_train_loader, binary)

1.0

In [19]:
compute_accuracy(binary_test_loader, binary)

1.0

In [22]:
compute_accuracy(multi_train_loader, multi)

0.9765625

In [21]:
compute_accuracy(multi_test_loader, multi)

0.9666666666666667